# Root finding and implicit ODEs

With the linear algebra techniques of solving coupled systems of equations we can now construct solutions to coupled ODEs. As an example we will write a nuclear network code. Such networks exhibit an ODE behaviour that is called _stiff_. Coefficients in stiff ODE systems have a very large range, which implies that the explicit solution scheme we have used so far would demand taking very small time steps to calculate details of the solutions that are not relevant. The solution is to adopt an implicit scheme. It involves numerical root-finding with the Newton-Raphson method. 

* Newton-Raphson root finding
* implicit solution of ODEs using Newton-Raphson
* solving coupled systems of ODEs

## Root finding: Newton-Raphson method

**Literature: ** Numerical Recipes, Chapter 9.0 and 9.4 

Root-finding is required in many applications of computational physics. It is at the core of the fundamental task to solve an equation numerically. Any equation with a right-hand and left-hand side can be rearranged to 
$$ f(x) = 0.$$
If the there is only one independent variable we have just a one-dimensional problem. The Newton-Raphson method is a robust and easy method to find the root of a function. It has very fast convergence if there is a good initial guess, but it has - like other methods - several pathologies that one needs to be aware off. One of its advantages is that it does generalize to multiple dimensions in a straight-forward way. 

### Mathematical idea of the method

The method starts with a good initial guess $x_{ini}$ of the root. Evaluating the function and the derivative for the initial guess provides a correction $\delta$ toward the actual root $x'$.

![nr-method](../images/NR_scheme.jpg)

We can then repeat this improvement until a termination criterion certifies that the solution is good enough. 

### Algorithm
Once we have the basic idea and mathematical formulation of the method in place, and before we actually start the coding it often is useful to write our strategy down in the form of an algorithm.

![nr-algorithm](../images/NR_algorithm.jpg)


### Termination condition

The termination conditions determine if a solution is good enough to
be accepted. Really, we are checking on the precision with which the
solution is satisfying the discretized equation. The termination
conditions for the NR method may include the following:

- $\delta < \delta_{lim}$: corrections are smaller than some limit
- the function evaluation is smaller than some limit, i.e. close to zero: $f(x_n)< \epsilon_{fmax}$ 

### Pathological cases

This termination condition can check on a number of things, and the
particular choices may depend on the particular problem. Before
writing down the termination conditions it makes sense to anticipate
pathological cases, i.e. situations in which this method will fail:

- higher-order terms are important, and initial guess is not good enough $\rightarrow$ the method will send you off to infinity
- non-convergent cycles: there could be a case, for example, when table look-up is involved in calculating the derivative, that correction are bouncing the guess from 
one side of the root to the other indefinitely without ever coming closer
- there could be multiple roots, and the one the NR method is finding is only a local root selected by the choice of initial guess.

Note that all methods have pathologies, or in other words, have their
particular weaknesses and strength. Selecting the most appropriate
method for a particular problem requires you to consider the
advantages and disadvantages of different methods.


The following remedies can be added to the termination conditions to accomodate the pathological cases:

- corrections are monotonically decreasing
- limit maximum number of iterations 
- restart iterations with different initial guess


### Example
Let's explore some of the pathologies that can be encountered with an example.

$$ f(x) = x^3 + 3x^2 -3 $$

In [ ]:
%pylab nbagg 

In [ ]:
# We will need the function 
f = lambda x: x**3 + 3*x**2 - 3

# and the derivative
def derivs(f,x,h):
    '''
    Return 1st order numerical derivative
    
    f : function
    x : x value to evaluate derivative
    h : interval in x for derivative
    '''
    
    dfdx = (f(x+h) - f(x))/h
    return dfdx
 

In [ ]:
# visual inspection
x = linspace(-3,1.5,40)
plot(x,f(x))
axhline(color='k',linestyle='dashed')

In [ ]:
h = 0.01
#xx = 0.6
xx = -0.2
xx = -0.2
#xx = -2.0
#xx = -2.01; h=0.01
plot(xx,f(xx),'v')

In [ ]:
x = xx
delta = -f(x)/derivs(f,x,h)
xx = x + delta
print(xx,f(xx))
plot(xx,f(xx),'h')